In [ ]:
import pandas as pd
import numpy as np
import random

BASE_PATH = "data/processed/non_wildfire/"

# Load the dataset
df = pd.read_csv('data/raw/mapdataall.csv')

# filtering rows related to wildfires
wildfire_data = df[df['incident_type'].str.contains('wildfire', case=False, na=False)]

# Save the filtered data to a new CSV file
wildfire_data.to_csv(BASE_PATH + 'wildfire_records(debug).csv', index=False)

print(f"Filtered wildfire data saved to 'wildfire_records.csv'.")


Filtered wildfire data saved to 'wildfire_records.csv'.


In [ ]:
wildfire_dates = pd.to_datetime(wildfire_data['incident_dateonly_created'], errors='coerce').dropna().unique()


# range of dates from 2010 to 2025-03-20
start_date = pd.to_datetime("2010-01-01")
end_date = pd.to_datetime("2025-03-20")
date_range = pd.date_range(start=start_date, end=end_date)

# Find the dates that did not have wildfire incidents
missing_dates = [date for date in date_range if date not in wildfire_dates]

# Randomly select twice the number of wildfire incidents from the missing dates
num_wildfire_incidents = len(wildfire_dates)
random_missing_dates = random.sample(missing_dates, 2 * num_wildfire_incidents)


# Randomly select twice the number of wildfire incidents from the missing dates
num_wildfire_incidents = len(wildfire_dates)
random_missing_dates = random.sample(missing_dates, 2 * num_wildfire_incidents)

random_missing_data = []

for missing_date in random_missing_dates:
    # Randomly select one wildfire incident to associate with the missing date
    random_wildfire = wildfire_data.sample(1)
    county = random_wildfire['incident_county'].values[0]
    latitude = random_wildfire['incident_latitude'].values[0]
    longitude = random_wildfire['incident_longitude'].values[0]

    # Add to the result list
    random_missing_data.append([missing_date, county, latitude, longitude])

# Convert to DataFrame and save to CSV
missing_df = pd.DataFrame(random_missing_data, columns=["date", "county", "latitude", "longitude"])

# Save to CSV file
missing_df.to_csv(BASE_PATH + 'non_wildfire_dates.csv', index=False)

print(f"Randomly selected missing dates saved to 'non_wildfire_dates.csv'.")

Randomly selected missing dates saved to 'non_wildfire_dates.csv'.


In [ ]:
num_rows = len(missing_df)
print(f"Total rows: {num_rows}")